# Stegasus

## Commons

### Colab Specific

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
!java -version

The system cannot find the path specified.
'update-alternatives' is not recognized as an internal or external command,
operable program or batch file.
openjdk version "17.0.7" 2023-04-18 LTS
OpenJDK Runtime Environment Corretto-17.0.7.7.1 (build 17.0.7+7-LTS)
OpenJDK 64-Bit Server VM Corretto-17.0.7.7.1 (build 17.0.7+7-LTS, mixed mode, sharing)


In [2]:
!git clone https://github.com/NasoohOlabi/Stegasus.git

^C


In [ ]:
!mv Stegasus/* .
!rm -r Stegasus

In [ ]:
%pip install -r requirements.txt

In [ ]:
ROOT_DIR = '.'

### General Common

In [1]:
#@title random_bit_stream
import random

def random_bit_stream(length=None):
    """Return a random string of zeros and ones of the given length (default: random integer between 0 and 100)."""
    if length is None:
        length = random.randint(0, 100)
    return ''.join(str(random.randint(0, 1)) for _ in range(length))
def int_to_binary_string(n: int, length: int):
    binary_str = bin(n)[2:]  # convert to binary string, remove '0b' prefix
    padded_str = binary_str.rjust(length, '0')  # pad with zeros to length
    return padded_str

In [2]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Stegasus Import

In [3]:
from stegasus import StegasusEncode, StegasusDecode

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros


Setting up Typoceros Completed



In [4]:
StegasusEncode('hi, how are you?','10101010010101')

('hi, how atre you?', '01010010101')

In [5]:
# FIXME
StegasusDecode('hi, how atre you?')

('hi, how are you?', '101')

In [6]:
LONG_TEXT = '''Metaphysical solipsism is a variety of solipsism. Based on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality and that all other realities, including the external world and other persons.'''

In [7]:
Famous_Demo = 'Hi, How are you?'

In [8]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' took {elapsed_time:.6f} seconds to execute.")
        return result
    return wrapper

In [9]:
# bench and test
@timer
def bench(xtx:str):
  data = random_bit_stream(len(xtx))
  print(f'original={xtx}')
  encoded,rem = StegasusEncode(xtx,data)
  org,deData = StegasusDecode(encoded)
  print(f'data={data[:-len(rem)]}')
  print(f'encoded={encoded}')
  print(f'capacity={100*((len(xtx) - len(rem)) / len(xtx))}%')

In [10]:
from SampleData import ConversationsRepo

In [11]:
for m in ConversationsRepo.get():
  print(m)
  bench(m)

 Hi! Do you like poetry?


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 31: invalid continuation byte

## Benchmarks

In [ ]:
from SampleData import ConversationsRepo
import random
from Bot import *

In [ ]:
def test_stream(minimum=1,maximum=ConversationsRepo.ConversationsCount):
  gened = set()
  for i in range(maximum - minimum):
    x = random.randint(minimum,maximum)
    while x in gened:
      x = random.randint(minimum,maximum)
    gened.add(x)
    yield x

In [ ]:
StegasusEncode("I sure do. You can listen to JUpiter's storms on AM radio",random_bit_stream(1000))

In [ ]:
for idx in test_stream():
  conversation = ConversationsRepo.get(idx)
  # chat = Chat(Person('Person','A',18,'male','Damascus','a@gmail.com',{},[]),Person('Person','B',18,'female','Damascus','b@gmail.com',{},[]),lambda x: x)
  people = [Person('Alice','A',18,'male','Damascus','a@gmail.com',{},[]),Person('Bob','B',18,'female','Damascus','b@gmail.com',{},[])]
  estimated_length = sum(map(len,conversation))
  messages = [Message(people[i%2],m) for i,m in enumerate(conversation)]
  chat = Chat(people[0],people[1],lambda x : x)
  chat.messages = messages
  print(f'idx={idx},payload size={estimated_length * 2}')
  chat.render()
  payload = random_bit_stream(estimated_length * 2)
  for idx in range(len(messages)):
    carrier_size = len(messages[idx].text)
    messages[idx].text, rem = StegasusEncode(messages[idx].text,payload)
    print(f'text length={carrier_size}\tencoded bits={len(payload)-len(rem)}\tratio={int(((len(payload)-len(rem))/carrier_size)*100)}%\ttext={messages[idx].text}')
    payload = rem
  chat.render()